In [1]:
import numpy as np
import time
import math
from numpy.linalg import qr
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
from scipy.linalg.interpolative import estimate_spectral_norm
from ipynb.fs.full.Traditional_PCA import do_pca
from sklearn.metrics import mean_squared_error

In [2]:
def memory_restricted_pca(B, X):
    """
    B = block size
    X = data matrix d x n, where n is the number of data points and d is the number of attributes
    output q = estimated eigen vectors of cov(X)
    """
    # get the number of attributes
    d = X.shape[0] 
    
    # create an identity matrix dxd
    I = np.identity(d) 
    
    # initialize orthogonal matrix q from random multivariate normal distribution
    q = np.random.normal(0, I) 
    
    # get spectral norm of q
    spectral_norm_q = estimate_spectral_norm(q)
    
    # update q
    q = q/spectral_norm_q
    
    # get the number of data point
    n = X.shape[1]
    
    index = 0 
    for i in range(int(n/B)):
        # initialize matrix s of size dxd
        s = np.zeros((d, d))
        for j in range(B):
            x = X[:,index].reshape((-1,1))
            ##s = s + ((1/B) * np.inner(q, x.T).dot(x.T))
            s = s + ((1/B) * x.dot(x.T).dot(q))
            index += 1
        q = s/estimate_spectral_norm(s)
    return q

In [3]:
np.random.seed(1)
rng = np.random.RandomState(1999)
d = 3
n = 15
X = rng.randn(d,n) + 2

In [4]:
B = 5 
q = memory_restricted_pca(B, X)

In [5]:
## compute PCA
Y = q.T.dot(X)
print(Y.shape)
## get reconstruction
X_est = q.dot(Y)
print(X_est.shape)
## get reconstruction error
mean_squared_error(X, X_est)
estimate_spectral_norm((X - X_est))

(3, 15)
(3, 15)


3.3310078960083445

In [6]:
## let's compare the memory restricted PCA with traditional PCA
M = np.mean(X.T, axis=0) # first pass on X
C = X.T - M             # second pass on X
Y, eig_vectors, k = do_pca(C) ## Y.T should be the same as reduced_X above
eig_vectors.shape
## compute PCA
Y = eig_vectors.T.dot(X)
print(Y.shape)
## get reconstruction
X_est = eig_vectors.dot(Y)
print(X_est.shape)
## get reconstruction error
##mean_squared_error(X, X_est)
estimate_spectral_norm((X - X_est))

(3, 15)
(3, 15)


1.282623750788516e-14

## Modified Memory Restricted PCA with Forgetting Factor

In [7]:
def update_q(X, d, q, B):
    print("q input is {}".format(q))
    s = np.zeros((d, d))
    for j in range(B):
        x = X[:,j].reshape((-1, 1))
        s = s + ((1/B) * x.dot(x.T).dot(q))
    q = s/estimate_spectral_norm(s)
    print("q output is {}".format(q))
    return q

In [8]:
def memory_restricted_pca_all(B, X, f):
    """
    Parameters
    ----------
    B : int 
        block size
    X : numpy array matrix
        data matrix d x n, where n is the number of data points and d is the number of attributes
    output q = estimated eigen vectors of cov(X)
    f : float 
        forgetting factor, f's value can be (0,1]
    """
    # get the number of attributes
    d = X.shape[0] 
    
    # create an identity matrix dxd
    I = np.identity(d) 
    
    # initialize orthogonal matrix q from random multivariate normal distribution
    q = np.random.normal(0, I) 
    print("init q {}".format(q))
    # get spectral norm of q
    spectral_norm_q = estimate_spectral_norm(q)
    
    # update q
    q = q/spectral_norm_q
    
    # get the number of data point
    n = X.shape[1]
    print("q {}".format(q))
    for i in range(n//B):
        X_window = X[:,i*B:(i + 1)*B]
        if i==0:
            q = update_q(X_window, d, q, B)
        else:
            ## applying forgetting factor before updating q
            q = q * math.pow(f,2)
            print("now calling update_q at i={}".format(i))
            q = update_q(X_window, d, q, B)
    return q

In [9]:
## Test when no forgetting factor (f=1)
np.random.seed(1)
rng = np.random.RandomState(1999)
d = 3
n = 15
X2 = rng.randn(d,n) + 2
B = 5 
q = memory_restricted_pca_all(B, X2, f=1)
q

init q [[1.62434536 0.         0.        ]
 [0.         0.86540763 0.        ]
 [0.         0.         0.3190391 ]]
q [[1.         0.         0.        ]
 [0.         0.53277317 0.        ]
 [0.         0.         0.19641088]]
q input is [[1.         0.         0.        ]
 [0.         0.53277317 0.        ]
 [0.         0.         0.19641088]]
q output is [[0.52741313 0.25822548 0.09815724]
 [0.48468184 0.27164119 0.09608449]
 [0.49975463 0.26063343 0.10213721]]
now calling update_q at i=1
q input is [[0.52741313 0.25822548 0.09815724]
 [0.48468184 0.27164119 0.09608449]
 [0.49975463 0.26063343 0.10213721]]
q output is [[0.61140028 0.31348647 0.11909431]
 [0.27883754 0.1448032  0.05459931]
 [0.56228273 0.28944645 0.10997421]]
now calling update_q at i=2
q input is [[0.61140028 0.31348647 0.11909431]
 [0.27883754 0.1448032  0.05459931]
 [0.56228273 0.28944645 0.10997421]]
q output is [[0.566332   0.29140367 0.11055903]
 [0.47093686 0.24240225 0.09195249]
 [0.47430113 0.24411248 0.09263

array([[0.566332  , 0.29140367, 0.11055903],
       [0.47093686, 0.24240225, 0.09195249],
       [0.47430113, 0.24411248, 0.09263136]])

In [10]:
## Test with forgetting factor (f=0.1)
np.random.seed(1)
rng = np.random.RandomState(1999)
d = 3
n = 15
X2 = rng.randn(d,n) + 2
B = 5 
q = memory_restricted_pca_all(B, X2, f=2)
q

init q [[1.62434536 0.         0.        ]
 [0.         0.86540763 0.        ]
 [0.         0.         0.3190391 ]]
q [[1.         0.         0.        ]
 [0.         0.53277317 0.        ]
 [0.         0.         0.19641088]]
q input is [[1.         0.         0.        ]
 [0.         0.53277317 0.        ]
 [0.         0.         0.19641088]]
q output is [[0.52741313 0.25822548 0.09815724]
 [0.48468184 0.27164119 0.09608449]
 [0.49975463 0.26063343 0.10213721]]
now calling update_q at i=1
q input is [[2.10965254 1.03290193 0.39262898]
 [1.93872737 1.08656476 0.38433797]
 [1.9990185  1.04253371 0.40854884]]
q output is [[0.61140028 0.31348647 0.11909431]
 [0.27883754 0.1448032  0.05459931]
 [0.56228273 0.28944645 0.10997421]]
now calling update_q at i=2
q input is [[2.44560111 1.25394586 0.47637724]
 [1.11535015 0.57921282 0.21839725]
 [2.24913092 1.15778581 0.43989684]]
q output is [[0.566332   0.29140367 0.11055903]
 [0.47093686 0.24240225 0.09195249]
 [0.47430113 0.24411248 0.09263

array([[0.566332  , 0.29140367, 0.11055903],
       [0.47093686, 0.24240225, 0.09195249],
       [0.47430113, 0.24411248, 0.09263136]])